In [ ]:
# Советы по использованию блокнотов в Google Colab см.
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline

Создание чат-бот в рамках обучения
================

**Author:** [Matthew Inkawhich](https://github.com/MatthewInkawhich)


В этом уроке мы рассмотрим забавный и интересный пример использования моделей повторяющейся последовательности-в-последовательность. Мы обучим простого чат-бота, используя сценарии фильмов из [Cornell Movie-Dialogs
Corpus](https://www.cs.cornell.edu/~cristian/Cornell_Movie-Dialogs_Corpus.html).

Разговорные модели являются горячей темой в исследованиях искусственного интеллекта. Чат-боты могут быть найдены в различных условиях, включая приложения обслуживания клиентов и онлайн-службы поддержки. Эти боты часто работают на основе моделей на основе поиска, которые выдают предопределенные ответы на вопросы определенных форм. В строго ограниченной области, такой как служба поддержки ИТ компании, этих моделей может быть достаточно, однако они недостаточно надежны для более общих случаев использования. Обучение машины вести осмысленный разговор с человеком в нескольких областях — это исследовательский вопрос, который далек от решения. В последнее время бум глубокого обучения позволил создать мощные генеративные модели, такие как Google [Neural
Conversational Model](https://arxiv.org/abs/1506.05869), что знаменует собой большой шаг к многодоменным генеративным разговорным моделям. В этом руководстве мы реализуем этот тип модели в PyTorch.

![](https://pytorch.org/tutorials/_static/img/chatbot/bot.png)

``` {.sourceCode .python}
> hello?
Bot: hello .
> where am I?
Bot: you re in a hospital .
> who are you?
Bot: i m a lawyer .
> how are you doing?
Bot: i m fine .
> are you my friend?
Bot: no .
> you're under arrest
Bot: i m trying to help you !
> i'm just kidding
Bot: i m sorry .
> where are you from?
Bot: san francisco .
> it's time for me to leave
Bot: i know .
> goodbye
Bot: goodbye .
```

**Основные моменты урока**

-   Управление загрузкой и предварительной обработкой набора данных [Cornell Movie-Dialogs
    Corpus](https://www.cs.cornell.edu/~cristian/Cornell_Movie-Dialogs_Corpus.html)
    dataset
-   Реализовать модель «последовательность-последовательность» с [Luong attention
    mechanism(s)](https://arxiv.org/abs/1508.04025)
-   Совместное обучение моделей кодера и декодера с использованием мини-пакетов
-   Реализовать модуль декодирования жадного поиска
-   Взаимодействуйте с обученным чат-ботом


Подготовка
============

Для начала загрузите ZIP-файл данных
[здесь](https://zissou.infosci.cornell.edu/convokit/datasets/movie-corpus/movie-corpus.zip)


In [ ]:
# and put in a ``data/`` directory under the current directory.
#
# After that, let’s import some necessities.
#

import torch
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
from io import open
import itertools
import math
import json


USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")

Загрузка и предварительная обработка данных
======================

Следующим шагом будет переформатирование нашего файла данных и загрузка данных в структуры, с которыми мы сможем работать.

[Cornell Movie-Dialogs
Corpus](https://www.cs.cornell.edu/~cristian/Cornell_Movie-Dialogs_Corpus.html)
представляет собой богатый набор данных диалогов персонажей фильмов:

-   220 579 диалогов между 10 292 парами персонажей фильмов
-   9035 персонажей из 617 фильмов
-   304 713 всего высказываний

Этот набор данных большой и разнообразный, и в нем наблюдается большое разнообразие языковых формальностей, временных периодов, настроений и т. д. Мы надеемся, что это разнообразие сделает нашу модель устойчивой ко многим формам входных данных и запросов.

Сначала мы рассмотрим некоторые строки нашего файла данных, чтобы увидеть исходный формат.


In [ ]:
corpus_name = "movie-corpus"
corpus = os.path.join("data", corpus_name)

def printLines(file, n=10):
    with open(file, 'rb') as datafile:
        lines = datafile.readlines()
    for line in lines[:n]:
        print(line)

printLines(os.path.join(corpus, "utterances.jsonl"))

Создать форматированный файл данных
==========================

Для удобства мы создадим красиво отформатированный файл данных, в котором каждая строка будет содержать *предложение запроса* , разделенное табуляцией , и пару *предложений ответа*.

Следующие функции облегчают анализ файла необработанных данных `utterances.jsonl`.

-   `loadLinesAndConversations` разбивает каждую строку файла на словарь строк с полями: `lineID`, `characterID`и текст, а затем группирует их в диалоги с полями: `conversationID`, `movieID`, и строки.
-   `extractSentencePairs` извлекает пары предложений из разговоров


In [ ]:
# Разбивает каждую строку файла для создания строк и разговоров
def loadLinesAndConversations(fileName):
    lines = {}
    conversations = {}
    with open(fileName, 'r', encoding='iso-8859-1') as f:
        for line in f:
            lineJson = json.loads(line)
            # Извлечение полей для линейного объекта
            lineObj = {}
            lineObj["lineID"] = lineJson["id"]
            lineObj["characterID"] = lineJson["speaker"]
            lineObj["text"] = lineJson["text"]
            lines[lineObj['lineID']] = lineObj

            # Извлечь поля для объекта разговора
            if lineJson["conversation_id"] not in conversations:
                convObj = {}
                convObj["conversationID"] = lineJson["conversation_id"]
                convObj["movieID"] = lineJson["meta"]["movie_id"]
                convObj["lines"] = [lineObj]
            else:
                convObj = conversations[lineJson["conversation_id"]]
                convObj["lines"].insert(0, lineObj)
            conversations[convObj["conversationID"]] = convObj

    return lines, conversations


# Извлекает пары предложений из разговоров
def extractSentencePairs(conversations):
    qa_pairs = []
    for conversation in conversations.values():
        # Перебрать все строки разговора
        for i in range(len(conversation["lines"]) - 1):  # Игнорируем последнюю строку (на нее нет ответа)
            inputLine = conversation["lines"][i]["text"].strip()
            targetLine = conversation["lines"][i+1]["text"].strip()
            # Отфильтровать неправильные образцы (если один из списков пуст)
            if inputLine and targetLine:
                qa_pairs.append([inputLine, targetLine])
    return qa_pairs

Теперь мы вызовем эти функции и создадим файл. Мы назовем это `formatted_movie_lines.txt`.


In [ ]:
# Определить путь к новому файлу
datafile = os.path.join(corpus, "formatted_movie_lines.txt")

delimiter = '\t'
# Убрать разделитель
delimiter = str(codecs.decode(delimiter, "unicode_escape"))

# Инициализировать dict строки и dict разговоры
lines = {}
conversations = {}
# Загрузка строк и разговоров
print("\nProcessing corpus into lines and conversations...")
lines, conversations = loadLinesAndConversations(os.path.join(corpus, "utterances.jsonl"))

# Записать новый CSV-файл
print("\nWriting newly formatted file...")
with open(datafile, 'w', encoding='utf-8') as outputfile:
    writer = csv.writer(outputfile, delimiter=delimiter, lineterminator='\n')
    for pair in extractSentencePairs(conversations):
        writer.writerow(pair)

# Распечатать образец строк
print("\nSample lines from file:")
printLines(datafile)

Данные по загрузке и обрезке
==================

Наша следующая задача — создать словарь и загрузить пары предложений запрос/ответ в память.

Обратите внимание, что мы имеем дело с последовательностями **слов** , которые не имеют неявного отображения на дискретное числовое пространство. Таким образом, мы должны создать его, отобразив каждое уникальное слово, которое мы встречаем в нашем наборе данных, на значение индекса.

Для этого мы определяем `Voc` класс, который хранит отображение слов в индексы, обратное отображение индексов в слова, количество каждого слова и общее количество слов. Класс предоставляет методы для добавления слова в словарь (`addWord`), добавления всех слов в предложение (`addSentence`)
и обрезки редко встречающихся слов (`trim`). Подробнее об обрезке позже.


In [ ]:
# Токены слов по умолчанию
PAD_token = 0  # Используется для заполнения коротких предложений padding-short-sentences
SOS_token = 1  # Токен начала предложения start-of-sentences
EOS_token = 2  # Токен конца предложения end-of-sentences

class Voc:
    def __init__(self, name):
        self.name = name
        self.trimmed = False
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3  # Count SOS, EOS, PAD

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] += 1

    # Удалить слова ниже определенного порога подсчета
    def trim(self, min_count):
        if self.trimmed:
            return
        self.trimmed = True

        keep_words = []

        for k, v in self.word2count.items():
            if v >= min_count:
                keep_words.append(k)

        print('keep_words {} / {} = {:.4f}'.format(
            len(keep_words), len(self.word2index), len(keep_words) / len(self.word2index)
        ))

        # Повторная инициализация словарей
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3 # Подсчет токенов по умолчанию

        for word in keep_words:
            self.addWord(word)

Теперь мы можем собрать наш словарь и пары предложений «запрос/ответ». Прежде чем мы будем готовы использовать эти данные, мы должны выполнить некоторые предварительная обработка. Во-первых, мы должны преобразовать строки Unicode в ASCII, используя
`unicodeToAscii`. Затем мы должны преобразовать все буквы в строчные и обрезать все небуквенные символы, за исключением основных знаков препинания
(`normalizeString`). Наконец, чтобы помочь в обучении сходимости, мы отфильтруем предложения, длина которых больше порогового `MAX_LENGTH` значения
(`filterPairs`).


In [ ]:
MAX_LENGTH = 10  # Максимальная длина предложения, которую следует учитывать

# Превратите строку Unicode в обычный ASCII благодаря
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Строчные буквы, обрезка и удаление небуквенных символов
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    s = re.sub(r"\s+", r" ", s).strip()
    return s

# Прочитайте пары запрос/ответ и верните объект voc.
def readVocs(datafile, corpus_name):
    print("Reading lines...")
    # Прочитайте файл и разбейте его на строки.
    lines = open(datafile, encoding='utf-8').\
        read().strip().split('\n')
    # Разбейте каждую строку на пары и нормализуйте
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]
    voc = Voc(corpus_name)
    return voc, pairs

# Возвращает значение True, если оба предложения в паре «p» находятся ниже порога MAX_LENGTH.
def filterPair(p):
    # Входные последовательности должны сохранять последнее слово для токена EOS.
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH

# Фильтруйте пары, используя условие filterPair.
def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

# Используя определенные выше функции, верните заполненный объект voc и список пар.
def loadPrepareData(corpus, corpus_name, datafile, save_dir):
    print("Start preparing training data ...")
    voc, pairs = readVocs(datafile, corpus_name)
    print("Read {!s} sentence pairs".format(len(pairs)))
    pairs = filterPairs(pairs)
    print("Trimmed to {!s} sentence pairs".format(len(pairs)))
    print("Counting words...")
    for pair in pairs:
        voc.addSentence(pair[0])
        voc.addSentence(pair[1])
    print("Counted words:", voc.num_words)
    return voc, pairs


# Загрузить/собрать voc и пары
save_dir = os.path.join("data", "save")
voc, pairs = loadPrepareData(corpus, corpus_name, datafile, save_dir)
# Распечатайте несколько пар для проверки.
print("\npairs:")
for pair in pairs[:10]:
    print(pair)

Другая тактика, которая полезна для достижения более быстрой сходимости во время обучения, — это удаление редко используемых слов из нашего словаря. Уменьшение пространства признаков также смягчит сложность функции, которую модель должна научиться аппроксимировать. Мы сделаем это в два этапа:

1)  Обрежьте слова, используемые ниже `MIN_COUNT` порогового значения, используя  `voc.trim` функцию.

2)  Отфильтруйте пары с сокращенными словами.


In [ ]:
MIN_COUNT = 3    # Минимальный порог количества слов для обрезки

def trimRareWords(voc, pairs, MIN_COUNT):
    # Обрезать слова, использованные меньше MIN_COUNT, из voc
    voc.trim(MIN_COUNT)
    # Отфильтровать пары с обрезанными словами
    keep_pairs = []
    for pair in pairs:
        input_sentence = pair[0]
        output_sentence = pair[1]
        keep_input = True
        keep_output = True
        # Проверить входное предложение
        for word in input_sentence.split(' '):
            if word not in voc.word2index:
                keep_input = False
                break
        # Проверить выходное предложение
        for word in output_sentence.split(' '):
            if word not in voc.word2index:
                keep_output = False
                break

        # Сохраняйте только пары, которые не содержат обрезанных слов во
        # входном или выходном предложении.
        if keep_input and keep_output:
            keep_pairs.append(pair)

    print("Trimmed from {} pairs to {}, {:.4f} of total".format(len(pairs), len(keep_pairs), len(keep_pairs) / len(pairs)))
    return keep_pairs


# Обрезать voc и пары
pairs = trimRareWords(voc, pairs, MIN_COUNT)

Подготовка данных для моделей
=======================

Хотя мы приложили немало усилий для подготовки и преобразования наших данных в хороший словарный объект и список пар предложений, наши модели в конечном итоге будут ожидать числовые тензоры torch в качестве входных данных. Один из способов подготовки обработанных данных для моделей можно найти в руководстве по
[seq2seq translation
tutorial](https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html).
В этом руководстве мы используем размер пакета 1, что означает, что все, что нам нужно сделать, это преобразовать слова в наших парах предложений в соответствующие им индексы из словаря и передать это моделям.

Однако если вы заинтересованы в ускорении обучения и/или хотите использовать возможности распараллеливания графического процессора, вам придется тренироваться с помощью мини-пакетов.

Использование мини-пакетов также означает, что мы должны помнить об изменении длины предложений в наших пакетах. Чтобы разместить предложения разных размеров в одном пакете, мы сделаем наш пакетный входной тензор формы
*(max\_length, batch\_size)*,  где предложения короче
*max\_length* будут дополнены нулями после *EOS\_token*.

Если мы просто преобразуем наши английские предложения в тензоры, преобразуя слова в их индексы (`indexesFromSentence`) и zero-pad, наш тензор будет иметь форму  *(batch\_size, max\_length)* а индексация первого измерения вернет полную последовательность по всем временным шагам. Однако нам нужно иметь возможность индексировать наш пакет по времени и по всем последовательностям в пакете. Поэтому мы транспонируем нашу входную форму пакета в
*(max\_length, batch\_size)*, так что индексация по первому измерению вернет временной шаг по всем предложениям в пакете. Мы обрабатываем эту транспозицию неявно в функции `zeroPadding`.

![](https://pytorch.org/tutorials/_static/img/chatbot/seq2seq_batches.png){.align-center}

Функция `inputVar` брабатывает процесс преобразования предложений в тензор, в конечном итоге создавая правильно сформированный тензор с нулевым дополнением. Она также возвращает тензор `lengths` для каждой из последовательностей в пакете, который будет передан нашему декодеру позже.

Функция  `outputVar` выполняет функцию, похожую на `inputVar`, но вместо возврата `lengths` тензора она возвращает бинарный тензор маски и максимальную целевую длину предложения. Бинарный тензор маски имеет ту же форму, что и выходной целевой тензор, но каждый элемент, который является
*PAD\_token* , равен 0, а все остальные равны 1.

`batch2TrainData` просто берет группу пар и возвращает входные и целевые тензоры, используя вышеупомянутые функции.


In [ ]:
def indexesFromSentence(voc, sentence):
    return [voc.word2index[word] for word in sentence.split(' ')] + [EOS_token]


def zeroPadding(l, fillvalue=PAD_token):
    return list(itertools.zip_longest(*l, fillvalue=fillvalue))

def binaryMatrix(l, value=PAD_token):
    m = []
    for i, seq in enumerate(l):
        m.append([])
        for token in seq:
            if token == PAD_token:
                m[i].append(0)
            else:
                m[i].append(1)
    return m

# Возвращает дополненный тензор и длину входной последовательности
def inputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    padVar = torch.LongTensor(padList)
    return padVar, lengths

# Возвращает дополненный тензор целевой последовательности,
#маску заполнения и максимальную целевую длину.
def outputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    max_target_len = max([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    mask = binaryMatrix(padList)
    mask = torch.BoolTensor(mask)
    padVar = torch.LongTensor(padList)
    return padVar, mask, max_target_len

# Возвращает все элементы для заданной партии пар.
def batch2TrainData(voc, pair_batch):
    pair_batch.sort(key=lambda x: len(x[0].split(" ")), reverse=True)
    input_batch, output_batch = [], []
    for pair in pair_batch:
        input_batch.append(pair[0])
        output_batch.append(pair[1])
    inp, lengths = inputVar(input_batch, voc)
    output, mask, max_target_len = outputVar(output_batch, voc)
    return inp, lengths, output, mask, max_target_len


# Пример проверки
small_batch_size = 5
batches = batch2TrainData(voc, [random.choice(pairs) for _ in range(small_batch_size)])
input_variable, lengths, target_variable, mask, max_target_len = batches

print("input_variable:", input_variable)
print("lengths:", lengths)
print("target_variable:", target_variable)
print("mask:", mask)
print("max_target_len:", max_target_len)

Определить модели
=============

Модель Seq2Seq
-------------

Мозг нашего чат-бота — это модель последовательность-в-последовательность (seq2seq). Цель модели seq2seq — взять последовательность переменной длины в качестве входных данных и вернуть последовательность переменной длины в качестве выходных данных с использованием модели фиксированного размера.

[Sutskever et al.](https://arxiv.org/abs/1409.3215) обнаружили, что, используя две отдельные рекуррентные нейронные сети вместе, мы можем выполнить эту задачу. Одна RNN действует как **encoder**,  который кодирует входную последовательность переменной длины в вектор контекста фиксированной длины. Теоретически этот вектор контекста (последний скрытый слой RNN) будет содержать семантическую информацию о предложении запроса, которое вводится боту. Вторая RNN является **decoder** , который принимает входное слово и вектор контекста и возвращает предположение для следующего слова в последовательности и скрытое состояние для использования в следующей итерации.

![](https://pytorch.org/tutorials/_static/img/chatbot/seq2seq_ts.png){.align-center}

Источник изображения:
<https://jeddy92.github.io/JEddy92.github.io/ts_seq2seq_intro/>


Encoder
=======

Кодер RNN перебирает входное предложение по одному токену (например, слову) за раз, на каждом временном шаге выводя вектор «выходного» и вектор «скрытого состояния». Затем вектор скрытого состояния передается на следующий временной шаг, в то время как выходной вектор записывается. Кодер преобразует контекст, который он видел в каждой точке последовательности, в набор точек в многомерном пространстве, который декодер будет использовать для генерации осмысленного вывода для данной задачи.

В основе нашего кодировщика лежит многослойный Gated Recurrent Unit, изобретенный [Cho et al.](https://arxiv.org/pdf/1406.1078v3.pdf) в 2014 году. Мы будем использовать двунаправленный вариант GRU, что означает, что по сути есть две независимые RNN: одна, которая получает входную последовательность в нормальном последовательном порядке, и другая, которая получает входную последовательность в обратном порядке. Выходы каждой сети суммируются на каждом временном шаге. Использование двунаправленного GRU даст нам преимущество кодирования как прошлых, так и будущих контекстов.

Двунаправленная RNN:

![](https://pytorch.org/tutorials/_static/img/chatbot/RNN-bidirectional.png){.align-center
width="70.0%"}

Источник изображения: <https://colah.github.io/posts/2015-09-NN-Types-FP/>

Обратите внимание, что `embedding` слой используется для кодирования индексов наших слов в пространстве признаков произвольного размера. Для наших моделей этот слой будет отображать каждое слово в пространстве признаков размером *hidden\_size*. При обучении эти значения должны кодировать семантическое сходство между словами с похожим значением.

Наконец, при передаче дополненного пакета последовательностей в модуль RNN мы должны упаковать и распаковать заполнение вокруг прохода RNN, используя
`nn.utils.rnn.pack_padded_sequence` и
`nn.utils.rnn.pad_packed_sequence` соответственно.

**График вычислений:**

> 1)  Преобразовать индексы слов в embedding.
> 2)  Упакуйте padded batch ("рамочный" пакет) последовательностей для модуля RNN.
> 3)  Прямой проход через сеть GRU (Gated Recurrent Unit).
> 4)  Распакуйте padding.
> 5)  Просуммируйте двунаправленные выходы GRU.
> 6)  Возвратите выходные данные и конечное скрытое состояние.

**Inputs:**

-   `input_seq`: пакет входных предложений; shape=*(max\_length,
    batch\_size)*
-   `input_lengths`: список длин предложений, соответствующих каждому предложению в пакете;  shape=*(batch\_size)*
-   `hidden`:  скрытое состояние;  shape=*(n\_layers x num\_directions,
    batch\_size, hidden\_size)*

**Outputs:**

-   `outputs`: выходные признаки из последнего скрытого слоя GRU (сумма двунаправленных выходов);
    (sum of bidirectional outputs); shape=*(max\_length, batch\_size,
    hidden\_size)*
-   `hidden`: обновленное скрытое состояние из GRU; shape=*(n\_layers x
    num\_directions, batch\_size, hidden\_size)*


In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, hidden_size, embedding, n_layers=1, dropout=0):
        super(EncoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embedding = embedding

        # Инициализировать GRU; для параметров input_size и Hidden_size установлено значение «hidden_size».
        #   потому что наш входной размер — это embedding слова с количеством функций == скрытый_размер
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers,
                          dropout=(0 if n_layers == 1 else dropout), bidirectional=True)

    def forward(self, input_seq, input_lengths, hidden=None):
        # Преобразование индексов слов в embedding
        embedded = self.embedding(input_seq)
        # Упаковать padded batch последовательности для RNN module
        packed = nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
        # Прямой проход через GRU
        outputs, hidden = self.gru(packed, hidden)
        # Распаковка padding
        outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs)
        # СУммирование bidirectional GRU outputs
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:]
        # Возвратный вывод и окончательное скрытое состояние
        return outputs, hidden

Decoder
=======

Декодер RNN генерирует предложение ответа по одному токену. Он использует векторы контекста кодировщика и внутренние скрытые состояния для генерации следующего слова в последовательности. Он продолжает генерировать слова, пока не выведет *EOS\_token*, представляющий конец предложения. Распространенная проблема с vanilla seq2seq decoder заключается в том, что если мы полагаемся исключительно на вектор контекста для кодирования смысла всей входной последовательности, то, скорее всего, у нас будет потеря информации. Это особенно актуально при работе с длинными входными последовательностями, что значительно ограничивает возможности нашего декодера.

Чтобы бороться с этим,  [Bahdanau et al.](https://arxiv.org/abs/1409.0473)
создали «механизм внимания», который позволяет декодеру обращать внимание на определенные части входной последовательности, а не использовать весь фиксированный контекст на каждом этапе.

На высоком уровне внимание вычисляется с использованием текущего скрытого состояния декодера и выходов кодера. Выходные веса внимания имеют ту же форму, что и входная последовательность, что позволяет нам умножать их на выходы кодера, давая нам взвешенную сумму, которая указывает на части выходов кодера, на которые следует обратить внимание. Рисунок [Sean
Robertson's](https://github.com/spro) описывает это очень хорошо:

![](https://pytorch.org/tutorials/_static/img/chatbot/attn2.png){.align-center}

[Luong et al.](https://arxiv.org/abs/1508.04025) улучшили наработки Бахданау и др., создав «Глобальное внимание». Ключевое отличие в том, что при «Глобальном внимании» мы учитываем все скрытые состояния кодировщика, в отличие от «Локального внимания» Бахданау и др., которое учитывает только скрытое состояние кодировщика из текущего временного шага. Другое отличие в том, что при «Глобальном внимании» мы вычисляем веса внимания или энергии, используя скрытое состояние декодера только из текущего временного шага. Расчет внимания Бахданау и др. требует знания состояния декодера из предыдущего временного шага. Кроме того, Луонг и др. предоставляют различные методы для вычисления энергий внимания между выходом кодировщика и выходом декодера, которые называются «функциями оценки»:

![](https://pytorch.org/tutorials/_static/img/chatbot/scores.png){.align-center
width="60.0%"}

где $h_t$ = текущее состояние целевого декодера и $\bar{h}_s$ = все состояния кодера.

В целом, механизм глобального внимания можно суммировать на следующем рисунке. Обратите внимание, что мы реализуем «Уровень внимания» как отдельный,  `nn.Module` назвыаемый `Attn`. Выход этого модуля — нормализованный тензор весов softmax с формой shape *(batch\_size, 1,
max\_length)*.

![](https://pytorch.org/tutorials/_static/img/chatbot/global_attn.png){.align-center
width="60.0%"}


In [ ]:
# Luong attention layer (Уровень внимания Луонга)
class Attn(nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        self.method = method
        if self.method not in ['dot', 'general', 'concat']:
            raise ValueError(self.method, "is not an appropriate attention method.")
        self.hidden_size = hidden_size
        if self.method == 'general':
            self.attn = nn.Linear(self.hidden_size, hidden_size)
        elif self.method == 'concat':
            self.attn = nn.Linear(self.hidden_size * 2, hidden_size)
            self.v = nn.Parameter(torch.FloatTensor(hidden_size))

    def dot_score(self, hidden, encoder_output):
        return torch.sum(hidden * encoder_output, dim=2)

    def general_score(self, hidden, encoder_output):
        energy = self.attn(encoder_output)
        return torch.sum(hidden * energy, dim=2)

    def concat_score(self, hidden, encoder_output):
        energy = self.attn(torch.cat((hidden.expand(encoder_output.size(0), -1, -1), encoder_output), 2)).tanh()
        return torch.sum(self.v * energy, dim=2)

    def forward(self, hidden, encoder_outputs):
        # Рассчитайте веса (энергии) внимания по данной методике.
        if self.method == 'general':
            attn_energies = self.general_score(hidden, encoder_outputs)
        elif self.method == 'concat':
            attn_energies = self.concat_score(hidden, encoder_outputs)
        elif self.method == 'dot':
            attn_energies = self.dot_score(hidden, encoder_outputs)

        # Транспонировать размеры max_length и Batch_size
        attn_energies = attn_energies.t()

        # Возвращает нормализованные оценки вероятности softmax (с добавленным измерением)
        return F.softmax(attn_energies, dim=1).unsqueeze(1)

Теперь, когда мы определили наш подмодуль внимания, мы можем реализовать фактическую модель декодера. Для декодера мы будем вручную подавать нашу партию по одному временному шагу за раз. Это означает, что наш встроенный тензор слов и выход GRU будут иметь форму shape *(1, batch\_size, hidden\_size)*.

**График вычислений:**

> - 1)  Получить embedding текущего входного слова.
> - 2)  Проход вперед через однонаправленный unidirectional GRU.
> - 3)  Рассчитайте веса внимания на основе текущих выходных данных GRU из (2).
> - 4)  Умножьте веса внимания на выходы кодера, чтобы получить новый вектор контекста «взвешенной суммы».
>     \"weighted sum\" context vector.
> - 5)  Concatenate Объединить взвешенный вектор контекста и выходные данные GRU  используя уравнение Луонга eq. 5.
> - 6)  Предскажите следующее слово, используя уравнение Луонга 6 (без softmax).
> - 7)  Возврат выходных данных и конечного скрытого состояния.

**Inputs:**

-   `input_step`: один временной шаг (одно слово) пакета входной последовательности;
    shape=*(1, batch\_size)*
-   `last_hidden`: последний скрытый слой GRU; shape=*(n\_layers x
    num\_directions, batch\_size, hidden\_size)*
-   `encoder_outputs`: выходные данные модели кодировщика; shape=*(max\_length,
    batch\_size, hidden\_size)*

**Outputs:**

-   `output`: нормализованный тензор softmax, дающий вероятности того, что каждое слово является правильным следующим словом в декодированной последовательности; shape=*(batch\_size, voc.num\_words)*
-   `hidden`: конечное скрытое состояние GRU;; shape=*(n\_layers x
    num\_directions, batch\_size, hidden\_size)*


In [ ]:
class LuongAttnDecoderRNN(nn.Module):
    def __init__(self, attn_model, embedding, hidden_size, output_size, n_layers=1, dropout=0.1):
        super(LuongAttnDecoderRNN, self).__init__()

        # Сохранить для справки
        self.attn_model = attn_model
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout

        # Определение слоев
        self.embedding = embedding
        self.embedding_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout))
        self.concat = nn.Linear(hidden_size * 2, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)

        self.attn = Attn(attn_model, hidden_size)

    def forward(self, input_step, last_hidden, encoder_outputs):
        # Примечание: мы выполняем этот шаг (слово) за раз.
        # Получить embedding текущего входного слова
        embedded = self.embedding(input_step)
        embedded = self.embedding_dropout(embedded)
        # Проход вперед unidirectional GRU
        rnn_output, hidden = self.gru(embedded, last_hidden)
        # Рассчитать веса внимания на основе текущих результатов GRU output
        attn_weights = self.attn(rnn_output, encoder_outputs)
        # Умножьте веса внимания на выходные данные кодера,
        # чтобы получить новый вектор контекста «взвешенной суммы».

        context = attn_weights.bmm(encoder_outputs.transpose(0, 1))
        # Объедините вектор взвешенного контекста и
        # выходные данные GRU, используя уравнение Луонга. 5
        rnn_output = rnn_output.squeeze(0)
        context = context.squeeze(1)
        concat_input = torch.cat((rnn_output, context), 1)
        concat_output = torch.tanh(self.concat(concat_input))
        # Предскажите следующее слово, используя уравнение Луонга. 6
        output = self.out(concat_output)
        output = F.softmax(output, dim=1)
        # Возвратный вывод и окончательное скрытое состояние
        return output, hidden

Определить процедуру обучения
=========================

Скрытая потеря
-----------

Поскольку мы имеем дело с пакетами дополненных последовательностей, мы не можем просто учитывать все элементы тензора при вычислении потерь. Мы определяем
`maskNLLLoss` вычисление наших потерь на основе выходного тензора нашего декодера, целевого тензора и бинарного тензора маски, описывающего заполнение целевого тензора. Эта функция потерь вычисляет среднее отрицательное логарифмическое правдоподобие элементов, которые соответствуют *1* в тензоре маски.


In [ ]:
def maskNLLLoss(inp, target, mask):
    nTotal = mask.sum()
    crossEntropy = -torch.log(torch.gather(inp, 1, target.view(-1, 1)).squeeze(1))
    loss = crossEntropy.masked_select(mask).mean()
    loss = loss.to(device)
    return loss, nTotal.item()

Одна итерация обучения
=========================

Функция `train` содержит алгоритм для одной итерации обучения (одной партии входных данных).

Мы воспользуемся парой хитрых приемов, которые помогут добиться конвергенции:



-   Первый трюк заключается в использовании **teacher forcing (принуждения учителя)**. Это означает, что при некоторой вероятности, установленной с помощью  `teacher_forcing_ratio`, мы используем текущее целевое слово в качестве следующего входа декодера, а не используем текущую догадку декодера. Этот метод действует как обучающие колеса для декодера, помогая в более эффективном обучении. Однако принуждение учителя может привести к нестабильности модели во время вывода, поскольку у декодера может не быть достаточного шанса по-настоящему создать свои собственные выходные последовательности во время обучения. Таким образом, мы должны помнить о том, как мы устанавливаем `teacher_forcing_ratio`, , и не обманываться быстрой сходимостью.

-   Второй трюк, который мы реализуем, — это **gradient clipping (отсечение градиента)**. Это часто используемый метод для борьбы с проблемой «взрывного градиента». По сути, отсечением или пороговой установкой градиентов до максимального значения мы предотвращаем экспоненциальный рост градиентов и либо переполнение (NaN), либо выход за пределы крутых обрывов в функции стоимости.

![](https://pytorch.org/tutorials/_static/img/chatbot/grad_clip.png){.align-center
width="60.0%"}

Источник изображения: Гудфеллоу и др. Глубокое обучение . 2016.
<https://www.deeplearningbook.org/>

**Последовательность операций:**

1.  Пропустить вперед (forward) весь входной пакет через кодер.
2.  Инициализируйте входы декодера как SOS\_token, а скрытое состояние — как конечное скрытое состояние кодера.
3.  Передавайте входную пакетную последовательность через декодер по одному временному шагу за раз.
4.  Если учитель принудительно нажимает: установите следующий вход декодера в качестве текущего целевого значения; в противном случае: установите следующий вход декодера в качестве текущего выхода декодера.
5.  Рассчитайте и аккумулируйте убытки.
6.  Выполнить обратное распространение ошибки.
7.  Градиенты обрезки Clip gradients.
8.  Обновите параметры модели кодера и декодера.


In [ ]:
def train(input_variable, lengths, target_variable, mask, max_target_len, encoder, decoder, embedding,
          encoder_optimizer, decoder_optimizer, batch_size, clip, max_length=MAX_LENGTH):

    # Нулевые градиенты
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    # Установить параметры устройства
    input_variable = input_variable.to(device)
    target_variable = target_variable.to(device)
    mask = mask.to(device)
    # Длины для упаковки RNN всегда должны находиться в ЦП.
    lengths = lengths.to("cpu")

    # Инициализировать переменные
    loss = 0
    print_losses = []
    n_totals = 0

    # Прямой проход через кодер
    encoder_outputs, encoder_hidden = encoder(input_variable, lengths)

    # Создайте начальный ввод декодера (начните с токенов SOS для каждого предложения)
    decoder_input = torch.LongTensor([[SOS_token for _ in range(batch_size)]])
    decoder_input = decoder_input.to(device)

    # Установите начальное скрытое состояние декодера
    # в окончательное скрытое состояние кодера.
    decoder_hidden = encoder_hidden[:decoder.n_layers]

    # Определите, используем ли мы учителя, форсирующего эту итерацию.
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    # Forward batch of sequences through decoder one time step at a time
    # Пересылать пакет последовательностей через декодер по одному временному шагу за раз.
    if use_teacher_forcing:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # Принуждение учителя: следующий ввод — текущая цель
            decoder_input = target_variable[t].view(1, -1)
            # Посчитайте и накопите loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal
    else:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # No teacher forcing: next input is decoder's own current output
            # Никакого принуждения учителя: следующим входом является собственный текущий выход декодера.
            _, topi = decoder_output.topk(1)
            decoder_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
            decoder_input = decoder_input.to(device)
            # Посчитайте и накопите loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal

    # Выполнить обратное распространение ошибки
    loss.backward()

    # Clip gradients: gradients are modified in place
    # Градиенты клипа: градиенты изменяются на месте.
    _ = nn.utils.clip_grad_norm_(encoder.parameters(), clip)
    _ = nn.utils.clip_grad_norm_(decoder.parameters(), clip)

    # Отрегулируйте вес модели
    encoder_optimizer.step()
    decoder_optimizer.step()

    return sum(print_losses) / n_totals

Итерации обучения
===================

Наконец, пришло время связать полную процедуру обучения с данными. Функция  `trainIters` fотвечает за запуск
`n_iterations` обучения с учетом переданных моделей, оптимизаторов, данных и т. д. Эта функция вполне понятна, так как мы уже проделали тяжелую работу с `train` функцией.

Стоит отметить, что при сохранении модели мы сохраняем tarball, содержащий кодер и декодер `state_dicts` (параметры), оптимизаторы  `state_dicts`, потери, итерации и т. д Сохранение модели таким образом даст нам максимальную гибкость с контрольной точкой. После загрузки контрольной точки мы сможем использовать параметры модели для запуска вывода или продолжить обучение с того места, где остановились.


In [ ]:
def trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer, embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size, print_every, save_every, clip, corpus_name, loadFilename):

    # Загрузка пакетов для каждой итерации
    training_batches = [batch2TrainData(voc, [random.choice(pairs) for _ in range(batch_size)])
                      for _ in range(n_iteration)]

    # Инициализации
    print('Initializing ...')
    start_iteration = 1
    print_loss = 0
    if loadFilename:
        start_iteration = checkpoint['iteration'] + 1

    # Тренировочный цикл
    print("Training...")
    for iteration in range(start_iteration, n_iteration + 1):
        training_batch = training_batches[iteration - 1]
        # Извлечь поля из пакета
        input_variable, lengths, target_variable, mask, max_target_len = training_batch

        # Запуск обучающей итерации с пакетной программой
        loss = train(input_variable, lengths, target_variable, mask, max_target_len, encoder,
                     decoder, embedding, encoder_optimizer, decoder_optimizer, batch_size, clip)
        print_loss += loss

        # Распечатать прогресс
        if iteration % print_every == 0:
            print_loss_avg = print_loss / print_every
            print("Iteration: {}; Percent complete: {:.1f}%; Average loss: {:.4f}".format(iteration, iteration / n_iteration * 100, print_loss_avg))
            print_loss = 0

        # Сохранить контрольную точку
        if (iteration % save_every == 0):
            directory = os.path.join(save_dir, model_name, corpus_name, '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size))
            if not os.path.exists(directory):
                os.makedirs(directory)
            torch.save({
                'iteration': iteration,
                'en': encoder.state_dict(),
                'de': decoder.state_dict(),
                'en_opt': encoder_optimizer.state_dict(),
                'de_opt': decoder_optimizer.state_dict(),
                'loss': loss,
                'voc_dict': voc.__dict__,
                'embedding': embedding.state_dict()
            }, os.path.join(directory, '{}_{}.tar'.format(iteration, 'checkpoint')))

Определить оценку
=================

После обучения модели мы хотим иметь возможность общаться с ботом самостоятельно. Во-первых, мы должны определить, как мы хотим, чтобы модель декодировала закодированный ввод.

Жадное декодирование
---------------

Жадное декодирование — это метод декодирования, который мы используем во время обучения, когда **NOT** using teacher forcing. Другими словами, для каждого временного шага мы просто выбираем слово с `decoder_output` наибольшим значением
softmax. Этот метод декодирования оптимален на уровне одного временного шага.

Для облегчения операции жадного декодирования мы определяем
`GreedySearchDecoder` класс. При запуске объект этого класса принимает входную последовательность (`input_seq`) формы shape *(input\_seq length, 1)*, a scalar
input length (`input_length`) tensor, и  `max_length` чтобы ограничить длину предложения ответа. Входное предложение оценивается с использованием следующего вычислительного графика:

**График вычислений:**

1.  Передача вперед входных данных через модель кодировщика.
2.  Подготовьте последний скрытый слой кодировщика в качестве первого скрытого входа для декодера.
3.  Инициализируйте первый вход декодера как SOS\_token.
4.  Инициализируйте тензоры для добавления декодированных слов.
5.  Итеративно декодируйте по одному токену слова за раз:
>
>     :   a)  Прямой проход через декодер.
>         b)  олучите наиболее вероятный токен слова и его оценку softmax.
>         c)  Запишите token and score.
>         d)  Подготовьте текущий токен для следующего входа декодера.
>
6.  Возвращайте коллекции словесных токенов и оценок (word tokens and scores).


In [ ]:
class GreedySearchDecoder(nn.Module):
    def __init__(self, encoder, decoder):
        super(GreedySearchDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, input_seq, input_length, max_length):
        # Forward input through encoder model
        # Прямой проход ввода через модель кодировщика
        encoder_outputs, encoder_hidden = self.encoder(input_seq, input_length)
        # Подготовьте последний скрытый слой кодера, который будет первым скрытым входом в декодер.
        decoder_hidden = encoder_hidden[:decoder.n_layers]
        # Инициализируйте вход декодера с помощью SOS_token
        decoder_input = torch.ones(1, 1, device=device, dtype=torch.long) * SOS_token
        # Инициализируйте тензоры для добавления декодированных слов в
        all_tokens = torch.zeros([0], device=device, dtype=torch.long)
        all_scores = torch.zeros([0], device=device)
        # Итеративно декодируйте один токен слова за раз
        for _ in range(max_length):
            # Forward pass through decoder
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_outputs)
            # Получите наиболее вероятный токен слова и его оценку softmax.
            decoder_scores, decoder_input = torch.max(decoder_output, dim=1)
            # Запишите token and score
            all_tokens = torch.cat((all_tokens, decoder_input), dim=0)
            all_scores = torch.cat((all_scores, decoder_scores), dim=0)
            # Подготовьте текущий токен для
            # следующего ввода декодера (добавьте измерение)
            decoder_input = torch.unsqueeze(decoder_input, 0)
        # Возвратить коллекцию word tokens and scores
        return all_tokens, all_scores

Оценка текста
================

Теперь, когда у нас определен метод декодирования, мы можем написать функции для оценки предложения входной строки. Функция  `evaluate` управляет низкоуровневым процессом обработки предложения входной строки. Сначала мы форматируем предложение как входную партию индексов слов *batch\_size==1*. Мы делаем это, преобразуя слова предложения в соответствующие им индексы и транспонируя измерения, чтобы подготовить тензор для наших моделей. Мы также создаем тензор, `lengths` tкоторый содержит длину нашего входного предложения. В этом случае `lengths` является скалярным, поскольку мы оцениваем только одно предложение за раз (batch\_size==1). Затем мы получаем тензор декодированного предложения ответа, используя наш `GreedySearchDecoder`
объект  (`searcher`). Наконец, мы преобразуем индексы ответа в слова и возвращаем список декодированных слов.

`evaluateInput` действует как пользовательский интерфейс для нашего чат-бота. При вызове появляется поле ввода текста, в котором мы можем ввести наше предложение запроса. После ввода нашего предложения ввода и нажатия *Enter*, наш текст нормализуется так же, как наши обучающие данные, и в конечном итоге подается в `evaluate` function to obtain a decoded output sentence. функцию для получения декодированного выходного предложения. Мы зацикливаем этот процесс, поэтому мы можем продолжать общаться с нашим ботом, пока не введем либо
"q" либо "quit".

Наконец, если введенное предложение содержит слово, которого нет в словаре, мы корректно справляемся с этой проблемой, выводя сообщение об ошибке и предлагая пользователю ввести другое предложение.


In [ ]:
def evaluate(encoder, decoder, searcher, voc, sentence, max_length=MAX_LENGTH):
    ### Форматировать входное предложение как пакет
    # words -> indexes
    indexes_batch = [indexesFromSentence(voc, sentence)]
    # Создать тензор длин
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    # Транспонируйте размеры партии в соответствии с ожиданиями моделей.
    input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)
    # Используйте подходящее устройство
    input_batch = input_batch.to(device)
    lengths = lengths.to("cpu")
    # Расшифровать предложение с помощью поисковика
    tokens, scores = searcher(input_batch, lengths, max_length)
    # indexes -> words
    decoded_words = [voc.index2word[token.item()] for token in tokens]
    return decoded_words


def evaluateInput(encoder, decoder, searcher, voc):
    input_sentence = ''
    while(1):
        try:
            # Получить входное предложение
            input_sentence = input('> ')
            # Проверьте, завершен ли этот случай
            if input_sentence == 'q' or input_sentence == 'quit': break
            # Нормализовать предложение
            input_sentence = normalizeString(input_sentence)
            # Оценить предложение
            output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)
            # Отформатируйте и распечатайте ответное предложение
            output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
            print('Bot:', ' '.join(output_words))

        except KeyError:
            print("Error: Encountered unknown word.")

Запуск Модели
=========

Наконец, пришло время запустить нашу модель!

Независимо от того, хотим ли мы обучить или протестировать модель чат-бота, мы должны инициализировать отдельные модели кодера и декодера. В следующем блоке мы задаем желаемые конфигурации, выбираем начать с нуля или задать контрольную точку для загрузки, а также создаем и инициализируем модели. Не стесняйтесь играть с различными конфигурациями моделей для оптимизации производительности.


In [ ]:
# Настройка моделей
model_name = 'cb_model'
attn_model = 'dot'
#``attn_model = 'general'``
#``attn_model = 'concat'``
hidden_size = 500
encoder_n_layers = 2
decoder_n_layers = 2
dropout = 0.1
batch_size = 64

# Set checkpoint to load from; set to None if starting from scratch
# Установите контрольную точку для загрузки;
# установите значение None, если начинаете с нуля
loadFilename = None
checkpoint_iter = 4000

Пример кода для загрузки из контрольной точки:

``` {.sourceCode .python}
loadFilename = os.path.join(save_dir, model_name, corpus_name,
                    '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size),
                    '{}_checkpoint.tar'.format(checkpoint_iter))
```


In [ ]:
# Загрузить модель, если указано ``loadFilename``.
if loadFilename:
    # Если загружается на ту же машину, на которой обучалась модель
    checkpoint = torch.load(loadFilename)
    # При загрузке модели, обученной на GPU в CPU
    #checkpoint = torch.load(loadFilename, map_location=torch.device('cpu'))
    encoder_sd = checkpoint['en']
    decoder_sd = checkpoint['de']
    encoder_optimizer_sd = checkpoint['en_opt']
    decoder_optimizer_sd = checkpoint['de_opt']
    embedding_sd = checkpoint['embedding']
    voc.__dict__ = checkpoint['voc_dict']


print('Building encoder and decoder ...')
# Инициализация embbeding слов
embedding = nn.Embedding(voc.num_words, hidden_size)
if loadFilename:
    embedding.load_state_dict(embedding_sd)
# Инициализация encoder & decoder models
encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, voc.num_words, decoder_n_layers, dropout)
if loadFilename:
    encoder.load_state_dict(encoder_sd)
    decoder.load_state_dict(decoder_sd)
# Используйте подходящее устройство
encoder = encoder.to(device)
decoder = decoder.to(device)
print('Models built and ready to go!')

Запуск обучения модели (Training)
============

Если вы хотите обучить модель, выполните следующий блок.

Сначала мы задаем параметры обучения, затем инициализируем наши оптимизаторы и, наконец, вызываем `trainIters` функцию для запуска наших итераций обучения.


In [ ]:
# Настройка обучения/оптимизации
clip = 50.0
teacher_forcing_ratio = 1.0
learning_rate = 0.0001
decoder_learning_ratio = 5.0
n_iteration = 4000
print_every = 1
save_every = 500

# Ensure dropout layers are in train mode
# Убедитесь, что выпадающие слои находятся в режиме обучения.
encoder.train()
decoder.train()

# Инициализировать оптимизаторы
print('Building optimizers ...')
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)
if loadFilename:
    encoder_optimizer.load_state_dict(encoder_optimizer_sd)
    decoder_optimizer.load_state_dict(decoder_optimizer_sd)

# Если у вас есть CUDA, настройте CUDA для вызова
for state in encoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()

for state in decoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()

# Запуск обучающих итераций
print("Starting Training!")
trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer,
           embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size,
           print_every, save_every, clip, corpus_name, loadFilename)

Выполнить оценку модели
==============

Чтобы пообщаться с вашей моделью, запустите следующий блок.


In [ ]:
# Set dropout layers to ``eval`` mode
# Установите выпадающие слои в режим ``eval``.
encoder.eval()
decoder.eval()

# Инициализировать модуль поиска
searcher = GreedySearchDecoder(encoder, decoder)

# Начните общение (для начала раскомментируйте и введите следующую строку)
# evaluateInput(encoder, decoder, searcher, voc)

Заключение
==========

На этом все, ребята. Поздравляю, теперь вы знаете основы построения генеративной модели чат-бота! Если вам интересно, вы можете попробовать настроить поведение чат-бота, настроив модель и параметры обучения, а также настроив данные, на которых вы обучаете модель.

Ознакомьтесь с другими руководствами, чтобы узнать больше о крутых приложениях глубокого обучения в PyTorch!
